<a href="https://colab.research.google.com/github/RaniereRamos/Consultoria/blob/master/AA_test_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img alt="B2W logo" height="50px" src="https://ri.b2w.digital/img/2013/logo.png" align="left" hspace="15px" vspace="0px"> 

<h1><font color = "#0097a7"><strong>Analysis of A/B Test Results of the New Buy Box Model<strong></font></h1>

# What is an A/B test?

**A/B testing** is a framework for you to <font color = "#0097a7">**test different ideas**</font> for how to improve upon an existing design, often a website. With A/B testing you're able to take a set of new ideas, <font color = "#0097a7">**test them with a new experiment**</font>, <font color = "#0097a7">**statistically analyze the results**</font> to confidently say which idea is better, update your website or app to <font color = "#0097a7">**use the winning idea**</font>, and then continue the cycle over again. 

Let's walk through a simplified set of steps with an experiment. For do that, let's say we run a shopping website.

We want to know if a **different recommendation algorithm** would result in more visitors clicking the **'BUY"** button.


*   This is also referred to as **"conversion rate"**. If someone clicks we say they "converted".
*   The **conversion rate** is generally the number of people who did an action (for example, clicked a button)<font color = "#0097a7">**(orders)**</font> divided by the number of people who went to the page <font color = "#0097a7">**(visits)**</font>.

**Notes:** In our case, to test this we need three conditions: 

*   One, a **control** (our current recommendation algorithm).
*   Two, a **treatment 1** (a new recommendation algorithm).
*   Three, a **treatment 2** (other new recommendation algorithm).

>***Our hypothesis is that a new recommendation algorithm will make people more likely to buy what they needed.***

## Variables
*   **Question:** Will changing the recommendation algorithm result in more visitors clicking the **"BUY"** button?
*   **Hypothesis:** Using a new recommendation algorithm result in more **"BUY"** clicks.
*   **Dependent variable:** Clicked **"BUY"** button or not.
*   **Independent variable:** Buy Box's new recommendation algorithm.

# Get the data

### Import libraries
Here, we import the necessary libraries used in this notebook.

In [ ]:
# Install libraries for a progress bar
!pip install --upgrade 'google-cloud-bigquery[bqstorage,pandas]'

### Provide your credentials to the runtime

#### Authentications
**Notes:** You need to user identification and project to be accessed in Big Query.

In [ ]:
# User authentication
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

In [2]:
from oauth2client.client import GoogleCredentials
credentials = GoogleCredentials.get_application_default()
import getpass

In [3]:
import google.auth
from google.cloud import bigquery
from google.cloud import bigquery_storage

# Explicitly create a credentials object. This allows you to use the same
# credentials for both the BigQuery and BigQuery Storage clients, avoiding
# unnecessary API calls to fetch duplicate authentication tokens.
credentials, your_project_id = google.auth.default(
    scopes=["https://www.googleapis.com/auth/cloud-platform"]
)

# Make clients.
projeto='utility-canto-251714' # project's name in Big Query

bqclient = bigquery.Client(credentials=credentials, project=projeto,)
bqstorageclient = bigquery_storage.BigQueryReadClient(credentials=credentials)

#### Enable data table display
Colab includes the ```google.colab.data_table``` package that can be used to display large pandas dataframes as an interactive data table. It can be enabled with:

In [4]:
%load_ext google.colab.data_table

## Import dataset
Load the data and make it available in the object.

### Use BigQuery via magics
**Notes:** The ```google.cloud.bigquery``` library also includes a magic command which runs a query and either displays the result or saves it to a variable as a ```Dataframe```.

In [5]:
##%%bigquery --project utility-canto-251714 df (I didn't get save query's results using alias. So I used another method)

query = f"""
WITH brand_name AS (
 SELECT
     ARRAY(SELECT * FROM UNNEST(["ACOM"])) AS arr
),
dates AS (
 SELECT
     DATE("2021-06-23") AS min_date,
     DATE("2021-07-04") AS max_date
),
sort AS (
 SELECT
     sk_timestamp AS sort_date,
     UPPER(brand) AS brand,
     bboxtoken,
     DATE(sk_timestamp) 
           || '-' || 
         EXTRACT(hour FROM sk_timestamp)
           || '-' || 
         EXTRACT(minute FROM sk_timestamp)
           || '-' || 
         EXTRACT(second FROM sk_timestamp) 
           || '-' || 
         UPPER(brand) 
           || '-' || 
         CASE WHEN contextregion IS NULL THEN 'NAO IDENTIFICADO' ELSE contextregion END
           || '-' ||
         CASE WHEN context_opn IS NULL THEN 'NAO IDENTIFICADO' ELSE context_opn END
           || '-' || 
         productid 
           || '-' || 
         isfinance
           || '-' || 
         isprime
           || '-' || 
         CASE WHEN salesSolution IS NULL THEN 'NAO IDENTIFICADO' ELSE salesSolution END AS token_geral,
     buyboxTestAB,
     contextregion,
     departmentid,
  FROM
     utility-canto-251714.raw_buybox.ordenacao_2021,
     dates
 WHERE
     DATE(sk_timestamp) BETWEEN min_date AND max_date
     AND UPPER(brand) IN (SELECT pair FROM brand_name, UNNEST(arr) AS pair)
),

sales_ AS (
 SELECT
     partition_date,
     buy_box_token,
     department_id,
     device_subtype,
     order_id,
     order_line_id
 FROM
     b2w-bee-analytics.evaluated_sales.sales,
     dates
 WHERE
     DATE(partition_date) BETWEEN min_date AND max_date
     AND UPPER(brand) IN (SELECT pair FROM brand_name, UNNEST(arr) AS pair)
     AND buy_box_token LIKE 'smartbuybox-acom-v2%'
     AND delivery_type = 'VENDA'
     AND high_value_flag = 'N'
     AND payment_status = 'APROVADO'
     AND sku_type = 'PRODUTO'
     AND department_id NOT IN ('9087')
)

SELECT
    DATE(sort_date) AS data,
    ord.brand,
    CASE 
        WHEN buyboxTestAB = 'control-abexperiment20210622' THEN 'Control'
        WHEN buyboxTestAB = 'treatment1-abexperiment20210622' THEN 'Treatment 1'
        ELSE 'Treatment 2'
    END AS experiment,
    ord.departmentid AS department_id,
    contextregion AS region,
    COUNT(DISTINCT token_geral) AS visits,
    COUNT(DISTINCT order_id) AS orders                     
FROM
    sales_ sales RIGHT OUTER JOIN sort ord
                               ON sales.buy_box_token = ord.bboxtoken
WHERE 
    buyboxTestAB IN ('control-abexperiment20210622','treatment1-abexperiment20210622','treatment2-abexperiment20210622')
GROUP BY 1,2,3,4,5
ORDER BY 1 
"""

# set you bqstorage_client as argument in the to_dataframe() method.
# i've also added the tqdm progress bar here so you get better insight
# into how long it's still going to take
results = (
    bqclient.query(query)
    .result()
    .to_dataframe(bqstorage_client=bqstorageclient))

## **Exploratory Data Analysis in Python Using Pandas**

In [6]:
import pandas as pd

### **Data cleaning**

#### Data dimension

In [45]:
results.shape

(618220, 7)

#### Dataframe contents

In [46]:
results.head(15)

,data,brand,experiment,department_id,region,visits,orders
0,2021-06-23,ACOM,Control,13,NORTHEAST_INTERIOR_2908_992908,7,0
1,2021-06-23,ACOM,Control,36,SP_CAPITAL_3501_935030,4447,127
2,2021-06-23,ACOM,Treatment 2,13,MG_CAPITAL_3101_931055,31,1
3,2021-06-23,ACOM,Treatment 2,9072,NORTHEAST_CAPITAL_2701_992701,113,0
4,2021-06-23,ACOM,Control,58,RJ_CAPITAL_3301_933023,153,6
5,2021-06-23,ACOM,Treatment 1,9112,RJ_CAPITAL_3301_933022,1156,13
6,2021-06-23,ACOM,Control,9073,NORTHEAST_INTERIOR_2602_992602,11,0
7,2021-06-23,ACOM,Control,35,MIDWEST_CAPITAL_5201,1,0
8,2021-06-23,ACOM,Treatment 2,9068,PR_CAPITAL_4101_994101,269,0
9,2021-06-23,ACOM,Control,9085,MIDWEST_INTERIOR_5301_995301,3,0


#### Check for missing values

In [ ]:
results.isnull().sum()

#### Replace missing values with "NAO IDENTIFICADA"

In [10]:
# Replace null values for 'NAO IDENTIFICADA' when variable region is NULL 
df = results.fillna('NAO IDENTIFICADA')

In [ ]:
# Check new results 
df.isnull().sum()

In [ ]:
df = df.drop(['brand'], axis=1)
df

#### Write to CSV file

In [13]:
# Write dataframe in a csv file
df.to_csv('aa_results.csv', index=False)

In [ ]:
# Check that file is write
! ls

In [ ]:
# Check the context file in bash
! cat aa_results.csv

### **Questions**

#### **Conditional selection**
In performing exploratory data analysis, it is important to be able to select subsets of data to perform analysis or comparisons.

**1. Which department the most visits per experiment?** Here, we will return the entire row.

In [50]:
experiment_results[experiment_results['visits'] == experiment_results['visits'].max()]

,data,experiment,department_id,region,visits,orders
22823,2021-06-23,Treatment 2,36,NAO IDENTIFICADA,810796,63


### **Exploratory Data Analysis**

#### Read data

In [33]:
experiment_results = pd.read_csv('aa_results.csv')

#### Displays the dataframe

In [ ]:
# Shows the first five lines and the last five
experiment_results

In [35]:
# Show entire dataframe (if we want to see more ...)
pd.set_option('display.max_rows', experiment_results.shape[0]+1)

#### Overview of data types of each columns in the dataframe

In [41]:
experiment_results.dtypes

data             object
experiment       object
department_id     int64
region           object
visits            int64
orders            int64
dtype: object

#### Show specific data types in dataframe

In [ ]:
# Show only numbers type
df.select_dtypes(include=['number'])

In [ ]:
# Show only objects type
df.select_dtypes(include=['object'])

In [ ]:
df.sort_values('visits', ascending = False).head(15)[['data','experiment','region','visits','orders']]

## **Exploratory Data Analysis in R Language Using dplyr**

### **Install packages**
Here, we install and load the necessary packages used in this notebook.

In [ ]:
%%R
# Install packages
install.packages("tidyverse")     # Packages for data science (dplyr, ggplot2, tidyr, tibble)

In [ ]:
%%R
# Library packages
library(tidyverse)

### **Get the Data**

In [16]:
# To enable the magics below and use R within Python
%load_ext rpy2.ipython

In [ ]:
%%R
# Read a csv.file and save results
abtest <- read_csv('aa_results.csv')

In [ ]:
%%R
# Check dataframe´s class
class(abtest)

In [ ]:
%%R 
# Show summary of statistics
summary(abtest)